# Welcome to Full Stack Machine Learning's Week 4 Project!

In the final week, you will return to the workflow you built last week on the [taxi dataset](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page). 

## Task 1: Deploy the champion
Use what you have learned in the last two weeks to make necessary modifications and to deploy your latest version of the `TaxiFarePrediction` flow to Argo. Use `--branch champion` to denote this deployment as the champion model.

### The Baseline Model

Use the code from Week 3 project.


In [103]:
%%writefile ./taxi_faire_linear_regression.py
from metaflow import FlowSpec, step, card, conda_base, current, Parameter, Flow, project, retry
from metaflow.cards import Markdown, Table, Image, Artifact

URL = "https://outerbounds-datasets.s3.us-west-2.amazonaws.com/taxi/latest.parquet"
DATETIME_FORMAT = "%Y-%m-%d %H:%M:%S"

# @trigger(events=["s3"])   # Disable trigger for this project.
@project(name="fullstack")  # <-- Add project
@conda_base(
    libraries={
        "pandas": "1.4.2",
        "pyarrow": "11.0.0",
        "scikit-learn": "1.1.2",
    }
)
class TaxiFarePrediction(FlowSpec):
    data_url = Parameter("data_url", default=URL)

    def transform_features(self, df):
        idx = (df.fare_amount > 0)
        idx &= (df.trip_distance <= 100)
        idx &= (df.trip_distance > 0)
        idx &= (df.tip_amount >= 0)
        df = df[idx]
        return df

    @step
    def start(self):
        import pandas as pd
        from sklearn.model_selection import train_test_split
        df = pd.read_parquet(self.data_url)
        self.df = self.transform_features(df)
        self.X = self.df["trip_distance"].values.reshape(-1, 1)
        self.y = self.df["total_amount"].values
        self.next(self.linear_model)

    @step
    def linear_model(self):
        "Fit a single variable, linear model to the data."
        from sklearn.linear_model import LinearRegression
        self.model_type = "LinearRegression"
        self.model = LinearRegression()
        self.next(self.validate)

    @card(type="corise")
    @retry(times=2) 
    @step
    def validate(self):
        from sklearn.model_selection import cross_val_score
        # Get CV scores
        self.scores = cross_val_score(self.model, self.X, self.y, cv=5)
        # We still need to fit the model
        self.model.fit(self.X, self.y)

        # Cards
        current.card.append(Markdown("# Taxi Fare Prediction Results"))
        current.card.append(Artifact(self.model_type, name="model_type"))
        current.card.append(Artifact(self.model, name="model"))
        current.card.append(Artifact(self.scores, name="CV scores"))

        self.next(self.end)

    # @card
    @step
    def end(self):
        print("Success!")


if __name__ == "__main__":
    TaxiFarePrediction()

Overwriting ./taxi_faire_linear_regression.py


### Deploy with --production --branch champion

In [104]:
%%capture cap --no-stderr
# Deploy Baseline Model (LinearRegression)
!python ./taxi_faire_linear_regression.py --environment=conda \
    --production --branch champion argo-workflows create

In [105]:
# Manually trigger the Baseline Model
!python ./taxi_faire_linear_regression.py --environment=conda \
    --production --branch champion argo-workflows trigger

Metaflow 2.10.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: fullstack, Branch: prod.champion
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Workflow fullstack.prod.champion.taxifareprediction triggered on Argo Workflows (run-id argo-fullstack.prod.champion.taxifareprediction-jb6wq).
See the run in the UI at https://ui-pw-527107953.outerbounds.dev/TaxiFarePrediction/argo-fullstack.prod.champion.taxifareprediction-jb6wq


## Task 2: Build the challenger
Develop a second model, by using the same `TaxiFarePrediction` architecture. Then, deploy the flow to Argo as the `--branch challenger`. 
<br>
<br>
Hint: Modify the `linear_model` step. 
<br>
Bonus: Write a paragraph summary of how you developed the second model and tested it before deploying the challenger flow. Let us know in Slack what you found challenging about the task? 

### Test the new model in Notebook

- Use X, y from the previous model.
- Check if the scores are better than the baseline model.

In [67]:
# Champ namespace
champ_namespace = "production:mfprj-cqkixzvdsy3tjqdh-0-ndkt"
namespace(champ_namespace)

# Retrieve the data from the lastest run
flow_name = "TaxiFarePrediction"
run = Flow(flow_name).latest_successful_run
X = run.data.X
y = run.data.y
print(f"- X.shape = {X.shape}, y.shape = {y.shape}")

- X.shape = (863296, 1), y.shape = (863296,)


In [58]:
from sklearn.ensemble import GradientBoostingRegressor

# Create a new model
model = GradientBoostingRegressor()
model.fit(X, y)
print(f"Overall score = {model.score(X, y)}")

Overall score = 0.9103405969815334


In [59]:
# Cross-validation
scores = cross_val_score(model, X, y, cv=5)
print(f"CV Score = {scores}")

CV Score = [0.90333453 0.91098014 0.90638686 0.91128225 0.90649317]


### New Model File

In [106]:
%%writefile ./taxi_faire_gradient_boosting_regressor.py
from metaflow import FlowSpec, step, card, conda_base, current, Parameter, Flow, project, retry
from metaflow.cards import Markdown, Table, Image, Artifact

URL = "https://outerbounds-datasets.s3.us-west-2.amazonaws.com/taxi/latest.parquet"
DATETIME_FORMAT = "%Y-%m-%d %H:%M:%S"

# @trigger(events=["s3"])   # Disable trigger for this project.
@project(name="fullstack")  # <-- Add project
@conda_base(
    libraries={
        "pandas": "1.4.2",
        "pyarrow": "11.0.0",
        "scikit-learn": "1.1.2",
    }
)
class TaxiFarePrediction(FlowSpec):
    data_url = Parameter("data_url", default=URL)

    def transform_features(self, df):
        idx = (df.fare_amount > 0)
        idx &= (df.trip_distance <= 100)
        idx &= (df.trip_distance > 0)
        idx &= (df.tip_amount >= 0)
        df = df[idx]
        return df

    @step
    def start(self):
        import pandas as pd
        df = pd.read_parquet(self.data_url)
        self.df = self.transform_features(df)
        self.X = self.df["trip_distance"].values.reshape(-1, 1)
        self.y = self.df["total_amount"].values
        self.next(self.gradient_boosting_regressor_model)

    @step
    def gradient_boosting_regressor_model(self):
        "Fit a single variable, linear model to the data."
        from sklearn.ensemble import GradientBoostingRegressor
        self.model_type = "GradientBoostingRegressor"
        self.model = GradientBoostingRegressor()
        self.next(self.validate)

    @card(type="corise")
    @retry(times=2) 
    @step
    def validate(self):
        from sklearn.model_selection import cross_val_score
        # Get CV scores
        self.scores = cross_val_score(self.model, self.X, self.y, cv=5)

        # Fit the model with all of the data
        self.model.fit(self.X, self.y)

        # Cards
        current.card.append(Markdown("# Taxi Fare Prediction Results"))
        current.card.append(Artifact(self.model_type, name="model_type"))
        current.card.append(Artifact(self.model, name="model"))
        current.card.append(Artifact(self.scores, name="CV scores"))

        self.next(self.end)

    # @card  # Default card
    @step
    def end(self):
        print("Success!")


if __name__ == "__main__":
    TaxiFarePrediction()

Overwriting ./taxi_faire_gradient_boosting_regressor.py


### Deploy with --production --branch challenger

In [107]:
%%capture cap --no-stderr
# Deploy the Challenger model
!python ./taxi_faire_gradient_boosting_regressor.py --environment=conda \
    --production --branch challenger argo-workflows create

In [108]:
# Trigger
!python ./taxi_faire_gradient_boosting_regressor.py --environment=conda \
    --production --branch challenger argo-workflows trigger

Metaflow 2.10.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: fullstack, Branch: prod.challenger
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Workflow fullstack.prod.challenger.taxifareprediction triggered on Argo Workflows (run-id argo-fullstack.prod.challenger.taxifareprediction-pwgcq).
See the run in the UI at https://ui-pw-527107953.outerbounds.dev/TaxiFarePrediction/argo-fullstack.prod.challenger.taxifareprediction-pwgcq


## Task 3: Analyze the results
Return to this notebook, and read in the results of the challenger and champion flow using the Metaflow Client API.
<br>

#### Questions
Q1. Does your model perform better on the metrics you selected? 

  * <font color='navy'>Yes, the challenger model has better metrics.</font>

<p>

Q2. Think about your day job, how would you go about assessing whether to roll forward the production "champion" to your new model? 
  - What gives you confidence one model is better than another?<font color='navy'>
    * The cross-validation scores give us some confidence that the model performance is kind of consistent.
    * Alternatively, we should have reserved some test data that are not used for modeling/CV.
    </font>
    
    <br />

  - What kinds of information do you need to monitor to get buy-in from stakeholders that model A is preferable to model B?  <font color='navy'>
    * A/B testing with real-time data may help to convince people that the new model is doing better with latest data.
    * The real-time performance metrics (e.g. considering the cost of running different models) may provide a better picture.
    * Also need to check the confidence level, that the improvement is not just a random noise.
    </font>

### Analysis Code

In [83]:

from metaflow import namespace, Flow

def analyze_results(model_namespace: str, flow_name:str = "TaxiFarePrediction"):
    ## Retrieve data
    namespace(model_namespace)
    run = Flow(flow_name).latest_successful_run
    print(f"- namespace = {model_namespace}")
    print(f"- model_type = {run.data.model_type}")
    print(f"- CV scores = {run.data.scores}")

    ## Test the model
    # model = run.data.model
    # X = run.data.X
    # y = run.data.y
    # print(f"- X.shape = {X.shape}, y.shape = {y.shape}")
    # pred = model.predict(X[0:5])
    # print(f"- Sample predictions = {pred}")
    # score = model.score(X, y)
    # print(f"- Overall score = {score}")



In [86]:
# The Baseline Model
champ_namespace = "production:mfprj-cqkixzvdsy3tjqdh-0-ndkt"
analyze_results(champ_namespace)

- namespace = production:mfprj-cqkixzvdsy3tjqdh-0-ndkt
- model_type = LinearRegression
- CV scores = [0.89223724 0.90081135 0.89633068 0.90003852 0.89430825]


In [87]:
# The Challenge Model
chanllenge_namespace = "production:mfprj-uj55bdalmyvezuhl-0-bixe"
analyze_results(chanllenge_namespace)

- namespace = production:mfprj-uj55bdalmyvezuhl-0-bixe
- model_type = GradientBoostingRegressor
- CV scores = [0.90373194 0.9106865  0.90650655 0.9111977  0.90464117]


## CONGRATULATIONS! 🎉✨🍾
If you made it this far, you have completed the Full Stack Machine Learning Corise course. 
We are so glad that you chose to learn with us, and hope to see you again in future courses. Stay tuned for more content and come join us in [Slack](http://slack.outerbounds.co/) to keep learning about Metaflow!